In [2]:
import numpy as np
import time
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, VoltageUnits, Edge, AcquisitionType, READ_ALL_AVAILABLE
from nidaqmx.constants import LogicFamily 
# TWO_POINT_FIVE_V, THREE_POINT_THREE_V, FIVE_V

from nidaqmx import stream_readers 

import matplotlib.pyplot as plt

import numpy as np

In [3]:
class DataAcquisition(object):
    chmap = dict()
    # example chmap, {"apd", "trig", }
    def __init__(self, 
        chmap,
        clock_rate
    ):
        self.chmap = chmap
        self.daqtype = AcquisitionType.FINITE # finite or continuouse sampling
        self.clock_rate = clock_rate
        self.clock_edge = Edge.RISING
        self.trig_edge = Edge.RISING

        # allocate tasks, readers, writers, buffers
        self.tasksconfig = dict()
        self.readtasks = dict()
        self.writetasks = dict()
        self.readers = dict()
        self.writers = dict()
        self.buffers = dict()

    # methods for pulse experiment------------------------------------------------------
    def config_readtask(self, name, sampling_rate, min_volt=-10.0, max_volt=10.0, buffersize=2048):
        try:
            # Create and configure an analog input voltage task
            task = nidaqmx.Task(name)
            channel = task.ai_channels.add_ai_voltage_chan(
                self.ch_ai,"",
                # TerminalConfiguration.RSE,
                TerminalConfiguration.DIFF,
                min_volt,max_volt,
                VoltageUnits.VOLTS
            )
            # Configure sample clock
            task.timing.cfg_samp_clk_timing(
                self.clock_rate,
                self.chmap["clock"],
                self.clock_edge,
                self.daqtype, 
                buffersize)
            task.timing.ai_conv_src = self.chmap["clock"]
            task.timing.ai_conv_active_edge = self.clock_edge

            # Configure start trigger
            read_trig = task.triggers.start_trigger
            read_trig.cfg_dig_edge_start_trig(self.chmap["trigger"],self.trig_edge)

            # Configure reader stream
            reader = stream_readers.AnalogSingleChannelReader(task.in_stream)
            # reader = stream_readers.AnalogMultiChannelReader(task.in_stream)
            reader.read_all_avail_samp  = True
            buffer = np.zeros(buffersize, dtype=np.float64, order='C')

            self.read_tasks[name] = task
            self.readers[name] = reader
            self.buffers[name] = buffer
        except Exception as excpt:
            print("fuck you")
            print(excpt)
            # logger.info(f'Error configuring DAQ. Please check your DAQ is connected and powered. Exception details: {type(excpt).__name__} {excpt}')
            
            # self.clear_preallo(name)
            del self.readtasks[name]
            del self.readers[name]
            del self.buffers[name]
        return task

    def readout(self):
        pass
    # ----------------------------------------------------------------------------------

    # methods for line scanning -------------------------------------------------------------
    def config_linescanner(self):
        pass
    def readscanline(self):
        pass

    # ----------------------------------------------------------------------------------

    def read(self, taskname, timeout=10):
        num_read = self.readers[taskname].read_many_sample(
                self.buffers[taskname],
                READ_ALL_AVAILABLE,
                timeout
            )
        return self.buffers[taskname]

    # def clear_preallo(self, taskname):
    #     del self.readtasks[taskname]
    #     del self.writetasks[taskname]
    #     del self.readers[taskname]
    #     del self.writers[taskname]
    #     del self.buffers[taskname]
        

    def clear_preallo_all(self):
        self.readtasks = dict()
        self.writetasks = dict()
        self.readers = dict()
        self.writers = dict()
        self.buffers = dict()

    def close_readtask(self, idx=0):
        self.read_tasks[idx].close()
        self.clear_preallo(idx)

    def close_readtask_all(self):
        for tk in self.read_tasks:
            tk.close()
        self.clear_preallo_all()

    

In [62]:
NI_ch_APD = "Dev1/ai16"
NI_ch_Clock = "Dev1/PFI8" # clock source
NI_ch_Trig = "Dev1/PFI9" # trigger source
NI_ch_Ax123_InABp = "/Dev1/port/line8:13"
NI_ch_Ax123_OutABp = "/Dev1/port/line14:19"
NI_chmap = dict(apd = NI_ch_APD, 
                clock = NI_ch_Clock, 
                trig = NI_ch_Trig, 
                pax123_in = NI_ch_Ax123_InABp, 
                pax123_out = NI_ch_Ax123_OutABp)

clock_rate = 500E3 # max 500kHz for DAQ NI6343 sampling
sampling_rate = # must be a factor of the clock rate such that clock_rate/sampling_rate is a integer


daq = DataAcquisition(chmap=NI_chmap, clock_rate,  sampling_rate)

In [6]:
np.copy(a)

array([5, 6, 7])

In [10]:
np.zeros(56, dtype=np.float64, order='C')

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])